In [21]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

In [69]:
def score_based_strategy(full_dataset_file, session_length=7, buy_threshold=0.55, sell_threshold=0.4):

    data = pd.read_csv(full_dataset_file, parse_dates=['datetime'])
    
    data['daily_return'] = data['Cumulative_Return']
    
    # Support and Resistance
    data['resistance'] = data['Close'].rolling(50).max()
    data['support'] = data['Close'].rolling(50).min()
    
    def calculate_score(row):
        # RSI Score (0 to 1, bullish if RSI < 70)
        rsi_score = max(0, (70 - row['RSI']) / 70)
        
        # ATR Score (0 to 1, lower ATR = better)
        max_atr = data['ATR'].rolling(50).max().iloc[-1]  # Max ATR in last 50 days
        atr_score = 1 - (row['ATR'] / max_atr) if max_atr > 0 else 0.5
        
        # SMA Score (0 or 1 based on trend)
        if ( row['SMA_5'] > row['SMA_20'] > row['SMA_200'] or
            row['SMA_20'] > row['SMA_5'] > row['SMA_200'] or
            row['SMA_20'] < row['SMA_200'] < row['SMA_5'] or 
            row['SMA_50'] > row['SMA_200']):
            sma_score = 1
        else:
            sma_score = 0
        

        # Support and Resistance Score (Closer to Support or Resistance = worse)
        
        peak_punish = 0  # Default: No penalty
        min_punish = 0   # Default: No penalty

        if row['resistance'] > 0 and row['Close'] <= row['resistance']:
            resistance_deviation = abs(row['Close'] - row['resistance']) / row['resistance']
            peak_punish = - (1 - min(resistance_deviation, 1))  # Range: [-1, 0]

        if row['support'] > 0 and row['Close'] >= row['support']:
            support_deviation = abs(row['Close'] - row['support']) / row['support']
            min_punish = - (1 - min(support_deviation, 1))  # Range: [-1, 0]
        
        support_resistance_score = 1 + (peak_punish + min_punish)  # Range: [-1, 1]

        # Weighted Final Score (0 to 1)
        total_score = (
            0.45 * rsi_score + 
            0.65 * atr_score + 
            0.6 * sma_score + 
            0.3 * support_resistance_score
        )
        return total_score
    
    data['score'] = data.apply(calculate_score, axis=1)
    
    # Generate signals
    data['signal'] = 0
    data.loc[data['score'] > buy_threshold, 'signal'] = 1  # Buy signal
    data.loc[data['score'] < sell_threshold, 'signal'] = -1  # Sell signal
    
    # Position management
    data['position'] = 0
    data['consecutive_sells'] = 0
    
    in_session = False
    for i in range(1, len(data)):
        if not in_session:
            if data.at[i, 'signal'] == 1:
                in_session = True
                data.at[i, 'position'] = 1
                data.at[i, 'consecutive_sells'] = 0
        else:
            if data.at[i, 'signal'] == -1:
                data.at[i, 'consecutive_sells'] = data.at[i-1, 'consecutive_sells'] + 1
            else:
                data.at[i, 'consecutive_sells'] = 0
                
            if (data.at[i, 'consecutive_sells'] >= 3) or \
               ((i - data[data['position'].diff() == 1].index[-1]) >= session_length):
                in_session = False
                data.at[i, 'position'] = 0
            else:
                data.at[i, 'position'] = 1
    
    # Calculate returns
    data['strategy_return'] = data['position'].shift(1) * data['daily_return']
    data['cumulative_strategy'] = data['strategy_return'].cumsum()
    data['perfect_return'] = data['daily_return'].where(data['daily_return'] > 0, 0)
    data['cumulative_perfect'] = data['perfect_return'].cumsum()
    
    # Performance metrics
    total_strategy = data['cumulative_strategy'].iloc[-1]
    total_perfect = data['cumulative_perfect'].iloc[-1]
    efficiency = total_strategy / total_perfect if total_perfect > 0 else 0

    data['correct_prediction'] = (
        ((data['daily_return'] < 0) & (data['position'] == 0)) |  # True Negative (avoided loss)
        ((data['daily_return'] > 0) & (data['position'] == 1))     # True Positive (captured gain)
    )

    accuracy = data['correct_prediction'].mean()

    y_true = (data['daily_return'] > 0).astype(int)  # 1 if gain, 0 if loss
    y_pred = data['position']                        # 1 if invested, 0 if cash

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    report = classification_report(y_true, y_pred, target_names=['Avoid Loss (0)', 'Capture Gain (1)'])

    # Print results

    print("\nSTRATEGY PERFORMANCE:")
    print(f"Accuracy: {accuracy:.2%}")
    print("\nConfusion Matrix:")
    print(cm)
    print(f"\nTrue Negatives (Avoid Loss Correctly): {tn}")
    print(f"False Positives (Invested but Lost): {fp}")
    print(f"False Negatives (Missed Gains): {fn}")
    print(f"True Positives (Captured Gains): {tp}")
    print("\nClassification Report:")
    print(report)

    print("\nSCORE-BASED TRADING STRATEGY")
    print(f"Parameters: Buy Threshold={buy_threshold}, Sell Threshold={sell_threshold}")
    print(f"Strategy Return: {total_strategy:.2f}")
    print(f"Perfect Return: {total_perfect:.2f}")
    print(f"Efficiency: {efficiency:.2%}")
    print(f"Days in Market: {data['position'].sum()}/{len(data)}")
    
    return data


In [85]:
# Run the strategy
result = score_based_strategy(
    '../data/joined/full_dataset.csv',
    session_length=14, 
    buy_threshold=0.46,
    sell_threshold=0.44
)


STRATEGY PERFORMANCE:
Accuracy: 52.73%

Confusion Matrix:
[[ 325  409]
 [1140 1403]]

True Negatives (Avoid Loss Correctly): 325
False Positives (Invested but Lost): 409
False Negatives (Missed Gains): 1140
True Positives (Captured Gains): 1403

Classification Report:
                  precision    recall  f1-score   support

  Avoid Loss (0)       0.22      0.44      0.30       734
Capture Gain (1)       0.77      0.55      0.64      2543

        accuracy                           0.53      3277
       macro avg       0.50      0.50      0.47      3277
    weighted avg       0.65      0.53      0.57      3277


SCORE-BASED TRADING STRATEGY
Parameters: Buy Threshold=0.46, Sell Threshold=0.44
Strategy Return: 441.06
Perfect Return: 974.40
Efficiency: 45.27%
Days in Market: 1812/3277


In [ ]:
def score_based_strategy(full_dataset_file, session_length=7, buy_threshold=0.55, sell_threshold=0.4):
    data = pd.read_csv(full_dataset_file, parse_dates=['datetime'])
    
    data['daily_return'] = data['Cumulative_Return']
    data['resistance'] = data['Close'].rolling(50).max()
    data['support'] = data['Close'].rolling(50).min()
    data['ATR_50max'] = data['ATR'].rolling(50, min_periods=1).max()
    data['RSI_prev3'] = data['RSI'].shift(3)  # For slope calculation
    
    def calculate_score(row):
        rsi_level = max(0, (70 - row['RSI']) / 70)
        rsi_slope = (row['RSI'] - row['RSI_prev3']) / 3 if not pd.isna(row['RSI_prev3']) else 0
        rsi_momentum = 0.5 + min(0.5, max(-0.5, rsi_slope/20))  # Scale slope to [-0.5,0.5] then shift
        rsi_score = 0.7 * rsi_level + 0.3 * rsi_momentum
        
        atr_score = 1 - (row['ATR'] / row['ATR_50max']) if row['ATR_50max'] > 0 else 0.5
        
        sma_score = 0
        if row['SMA_5'] > row['SMA_20']:
            sma_score += 0.4 * min(1.0, (row['SMA_5'] - row['SMA_20']) / row['SMA_20'])
        if row['SMA_20'] > row['SMA_200']:
            sma_score += 0.6 * min(1.0, (row['SMA_20'] - row['SMA_200']) / row['SMA_200'])
        sma_score = min(1.0, sma_score)
        
        range_size = row['resistance'] - row['support']
        if range_size > 0:
            price_position = (row['Close'] - row['support']) / range_size
            support_resistance_score = 1 - 4 * (price_position - 0.5)**2
        else:
            support_resistance_score = 0.5  # Neutral if no clear range
            
        total_score = (
            0.4 * rsi_score + 
            0.45 * atr_score + 
            0.55 * sma_score + 
            0.2 * support_resistance_score
        )
        return total_score
    
    data['score'] = data.apply(calculate_score, axis=1)
    
    data['signal'] = 0
    data.loc[data['score'] > buy_threshold, 'signal'] = 1
    data.loc[data['score'] < sell_threshold, 'signal'] = -1
    
    data['position'] = 0
    hold_days = 0
    consecutive_sells = 0
    
    for i in range(1, len(data)):
        if data.at[i, 'position'] == 0:
            if data.at[i, 'signal'] == 1:
                data.at[i, 'position'] = 1
                hold_days = 1
                consecutive_sells = 0
        else:
            hold_days += 1
            
            if data.at[i, 'signal'] == -1:
                consecutive_sells += 1
            else:
                consecutive_sells = max(0, consecutive_sells - 0.5)
                
            if (hold_days >= session_length or 
                consecutive_sells >= 2 or 
                (data.at[i, 'daily_return'] > 0 and consecutive_sells >= 1)):
                data.at[i, 'position'] = 0
                consecutive_sells = 0
            else:
                data.at[i, 'position'] = 1
    
    # Calculate returns
    data['strategy_return'] = data['position'].shift(1) * data['daily_return']
    data['cumulative_strategy'] = data['strategy_return'].cumsum()
    data['perfect_return'] = data['daily_return'].where(data['daily_return'] > 0, 0)
    data['cumulative_perfect'] = data['perfect_return'].cumsum()
    
    # Performance metrics
    total_strategy = data['cumulative_strategy'].iloc[-1]
    total_perfect = data['cumulative_perfect'].iloc[-1]
    efficiency = total_strategy / total_perfect if total_perfect > 0 else 0

    data['correct_prediction'] = (
        ((data['daily_return'] < 0) & (data['position'] == 0)) |  # True Negative (avoided loss)
        ((data['daily_return'] > 0) & (data['position'] == 1))     # True Positive (captured gain)
    )

    accuracy = data['correct_prediction'].mean()

    y_true = (data['daily_return'] > 0).astype(int)  # 1 if gain, 0 if loss
    y_pred = data['position']                        # 1 if invested, 0 if cash

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    report = classification_report(y_true, y_pred, target_names=['Avoid Loss (0)', 'Capture Gain (1)'])

    # Print results

    print("\nSTRATEGY PERFORMANCE:")
    print(f"Accuracy: {accuracy:.2%}")
    print("\nConfusion Matrix:")
    print(cm)
    print(f"\nTrue Negatives (Avoid Loss Correctly): {tn}")
    print(f"False Positives (Invested but Lost): {fp}")
    print(f"False Negatives (Missed Gains): {fn}")
    print(f"True Positives (Captured Gains): {tp}")
    print("\nClassification Report:")
    print(report)

    print("\nSCORE-BASED TRADING STRATEGY")
    print(f"Parameters: Buy Threshold={buy_threshold}, Sell Threshold={sell_threshold}")
    print(f"Strategy Return: {total_strategy:.2f}")
    print(f"Perfect Return: {total_perfect:.2f}")
    print(f"Efficiency: {efficiency:.2%}")
    print(f"Days in Market: {data['position'].sum()}/{len(data)}")
    
    return data

# Run with optimized thresholds
result = score_based_strategy(
    '../data/joined/full_dataset.csv',
    session_length=10, 
    buy_threshold=0.33,
    sell_threshold=0.32
)


STRATEGY PERFORMANCE:
Accuracy: 56.06%

Confusion Matrix:
[[ 282  452]
 [ 988 1555]]

True Negatives (Avoid Loss Correctly): 282
False Positives (Invested but Lost): 452
False Negatives (Missed Gains): 988
True Positives (Captured Gains): 1555

Classification Report:
                  precision    recall  f1-score   support

  Avoid Loss (0)       0.22      0.38      0.28       734
Capture Gain (1)       0.77      0.61      0.68      2543

        accuracy                           0.56      3277
       macro avg       0.50      0.50      0.48      3277
    weighted avg       0.65      0.56      0.59      3277


SCORE-BASED TRADING STRATEGY
Parameters: Buy Threshold=0.33, Sell Threshold=0.32
Strategy Return: 576.95
Perfect Return: 974.40
Efficiency: 59.21%
Days in Market: 2007/3277


In [120]:
def score_based_strategy(full_dataset_file, buy_threshold=0.55, sell_threshold=0.4):
    data = pd.read_csv(full_dataset_file, parse_dates=['datetime'])
    
    data['daily_return'] = data['Cumulative_Return']
    data['resistance'] = data['Close'].rolling(50).max()
    data['support'] = data['Close'].rolling(50).min()
    data['ATR_50max'] = data['ATR'].rolling(50, min_periods=1).max()
    data['RSI_prev3'] = data['RSI'].shift(3) 
    
    def calculate_score(row):
        rsi_level = max(0, (70 - row['RSI']) / 70)
        rsi_slope = (row['RSI'] - row['RSI_prev3']) / 3 if not pd.isna(row['RSI_prev3']) else 0
        rsi_momentum = 0.5 + min(0.5, max(-0.5, rsi_slope/20))  # Scale slope to [-0.5,0.5] then shift
        rsi_score = 0.7 * rsi_level + 0.3 * rsi_momentum
        
        atr_score = 1 - (row['ATR'] / row['ATR_50max']) if row['ATR_50max'] > 0 else 0.5
        
        sma_score = 0
        if row['SMA_5'] > row['SMA_20']:
            sma_score += 0.4 * min(1.0, (row['SMA_5'] - row['SMA_20']) / row['SMA_20'])
        if row['SMA_20'] > row['SMA_200']:
            sma_score += 0.6 * min(1.0, (row['SMA_20'] - row['SMA_200']) / row['SMA_200'])
        sma_score = min(1.0, sma_score)
        
        # Optimized Support/Resistance
        range_size = row['resistance'] - row['support']
        if range_size > 0:
            price_position = (row['Close'] - row['support']) / range_size
            support_resistance_score = 1 - 4 * (price_position - 0.5)**2
        else:
            support_resistance_score = 0.5
            
        total_score = (
            0.4 * rsi_score + 
            0.45 * atr_score + 
            0.55 * sma_score + 
            0.2 * support_resistance_score
        )
        return total_score
    
    data['score'] = data.apply(calculate_score, axis=1)
    
    # Generate signals
    data['signal'] = 0
    data.loc[data['score'] > buy_threshold, 'signal'] = 1  # Buy signal
    data.loc[data['score'] < sell_threshold, 'signal'] = -1  # Sell signal
    
    # Simplified Position Management - Immediate entry/exit
    data['position'] = 0
    current_position = 0
    
    for i in range(1, len(data)):
        if current_position == 0:
            if data.at[i, 'signal'] == 1:
                current_position = 1
        else:
            if data.at[i, 'signal'] == -1:
                current_position = 0
                
        data.at[i, 'position'] = current_position
    
    # Calculate returns
    data['strategy_return'] = data['position'].shift(1) * data['daily_return']
    data['cumulative_strategy'] = data['strategy_return'].cumsum()
    data['perfect_return'] = data['daily_return'].where(data['daily_return'] > 0, 0)
    data['cumulative_perfect'] = data['perfect_return'].cumsum()
    
    # Performance metrics
    total_strategy = data['cumulative_strategy'].iloc[-1]
    total_perfect = data['cumulative_perfect'].iloc[-1]
    efficiency = total_strategy / total_perfect if total_perfect > 0 else 0

    data['correct_prediction'] = (
        ((data['daily_return'] < 0) & (data['position'] == 0)) |  # True Negative (avoided loss)
        ((data['daily_return'] > 0) & (data['position'] == 1))     # True Positive (captured gain)
    )

    accuracy = data['correct_prediction'].mean()

    y_true = (data['daily_return'] > 0).astype(int)  # 1 if gain, 0 if loss
    y_pred = data['position']                        # 1 if invested, 0 if cash

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    report = classification_report(y_true, y_pred, target_names=['Avoid Loss (0)', 'Capture Gain (1)'])

    # Print results

    print("\nSTRATEGY PERFORMANCE:")
    print(f"Accuracy: {accuracy:.2%}")
    print("\nConfusion Matrix:")
    print(cm)
    print(f"\nTrue Negatives (Avoid Loss Correctly): {tn}")
    print(f"False Positives (Invested but Lost): {fp}")
    print(f"False Negatives (Missed Gains): {fn}")
    print(f"True Positives (Captured Gains): {tp}")
    print("\nClassification Report:")
    print(report)

    print("\nSCORE-BASED TRADING STRATEGY")
    print(f"Parameters: Buy Threshold={buy_threshold}, Sell Threshold={sell_threshold}")
    print(f"Strategy Return: {total_strategy:.2f}")
    print(f"Perfect Return: {total_perfect:.2f}")
    print(f"Efficiency: {efficiency:.2%}")
    print(f"Days in Market: {data['position'].sum()}/{len(data)}")
    
    return data
# Run with optimized thresholds
result = score_based_strategy(
    '../data/joined/full_dataset.csv',
    buy_threshold=0.33,
    sell_threshold=0.31
)


STRATEGY PERFORMANCE:
Accuracy: 57.89%

Confusion Matrix:
[[ 259  475]
 [ 905 1638]]

True Negatives (Avoid Loss Correctly): 259
False Positives (Invested but Lost): 475
False Negatives (Missed Gains): 905
True Positives (Captured Gains): 1638

Classification Report:
                  precision    recall  f1-score   support

  Avoid Loss (0)       0.22      0.35      0.27       734
Capture Gain (1)       0.78      0.64      0.70      2543

        accuracy                           0.58      3277
       macro avg       0.50      0.50      0.49      3277
    weighted avg       0.65      0.58      0.61      3277


SCORE-BASED TRADING STRATEGY
Parameters: Buy Threshold=0.33, Sell Threshold=0.31
Strategy Return: 597.14
Perfect Return: 974.40
Efficiency: 61.28%
Days in Market: 2113/3277
